## Data for Doodle Scheduling problem

In [1]:
using JuMP, NamedArrays

availability =
  [ 0 0 1 1 0 0 0 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 0 1 1 0 1 1 1 1
    0 0 0 1 1 1 1 1 1 1 1 1 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    0 1 1 0 0 0 0 0 1 1 0 0 0
    0 0 0 1 1 1 1 0 0 0 0 0 0
    1 1 0 0 0 0 0 0 0 0 1 1 1
    1 1 1 0 0 0 0 0 0 1 1 0 0
    0 0 0 0 0 0 0 1 1 0 0 0 0
    0 0 0 0 0 0 1 1 1 0 0 0 0
    1 1 0 0 0 1 1 1 1 0 0 1 1
    1 1 1 0 1 1 0 0 0 0 0 1 1
    0 1 1 1 0 0 0 0 0 0 0 0 0
    1 1 0 0 1 1 0 0 0 0 0 0 0 ]

TIMES = ["10:00","10:20","10:40","11:00","11:20","11:40","lunch","1:00","1:20","1:40","2:00","2:20","2:40"]
NAMES = [:Manuel,:Luca,:Jule,:Michael,:Malte,:Chris,:Spyros,:Mirjam,:Matt,:Florian,:Josep,:Joel,:Tom,:Daniel,:Anne ]
times = NamedArray( availability, (NAMES,TIMES), ("NAME","TIME"))

15×13 Named Array{Int64,2}
NAME ╲ TIME │ 10:00  10:20  10:40  11:00  …   1:40   2:00   2:20   2:40
────────────┼──────────────────────────────────────────────────────────
:Manuel     │     0      0      1      1  …      0      0      0      0
:Luca       │     0      1      1      0         1      0      0      0
:Jule       │     0      0      0      1         1      1      1      1
:Michael    │     0      0      0      1         1      1      1      0
:Malte      │     0      0      0      0         0      0      0      0
:Chris      │     0      1      1      0         1      0      0      0
:Spyros     │     0      0      0      1         0      0      0      0
:Mirjam     │     1      1      0      0         0      1      1      1
:Matt       │     1      1      1      0         1      1      0      0
:Florian    │     0      0      0      0         0      0      0      0
:Josep      │     0      0      0      0         0      0      0      0
:Joel       │     1      1      0    

In [27]:
times[:Luca,"10:20"]

1

In [75]:
times[2,2]

1

In [72]:
for t in TIMES
    for n in NAMES
        print(times[n, t])
    end
    println()
end

000000011001101
010001011001111
110001001000110
101100100000010
001100100000101
000100100001101
001110100011000
101110000111000
110111000111000
011101001000000
001100011000000
001100010001100
001000010001100


In [84]:
using JuMP, Clp
m = Model(solver=ClpSolver())

@variable(m, assigned[1:size(TIMES,1)])
@variable(m, interviewer[NAMES] <= 1) # Each interviewer can only interview once

for t in 1:size(TIMES,1)
    for n in 1:size(NAMES,1)
        if(times[n][t] != 0)
            # Do something
        end
    end
end

for i in 1:size(TIMES,1)
    if(i == 7)
        @constraint(m, assigned[i] == 3)
    else
        @constraint(m, assigned[i] == 1)
    end
end

@objective(m, Max, sum(assigned))

solve(m)
println()
println(getobjectivevalue(m))


15.0


## Data for stadium building problem

In [2]:
using JuMP

tasks = 1:18
durations = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1]
predecessors = ( [], [1], [2], [2], [3], [4,5], [4], [6], [4,6], [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17] )
pred_dict = Dict(zip(tasks,predecessors));   # dictionary mapping tasks --> predecessors.

# additional columns of data (maximum reduction possible )
max_reduction =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0]  # max reduction (in weeks)
cost_reduction = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]  # cost of reduction ($1,000/week)
bonus_amount = 30     # bonus for expediting the project ($1,000/week )
;